# Binary Brains Team

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [2]:
df = pd.read_csv('Datasets/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [17]:
X_test.shape

(2000, 13)

In [18]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [20]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.4885 - accuracy: 0.7959
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4574 - accuracy: 0.8018
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4424 - accuracy: 0.8096
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.8120
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4287 - accuracy: 0.8125
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4273 - accuracy: 0.8159
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4247 - accuracy: 0.8142
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.8159
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.8179
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4157 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.3352 - accuracy: 0.8619
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3354 - accuracy: 0.8646
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3335 - accuracy: 0.8619
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3347 - accuracy: 0.8620
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3329 - accuracy: 0.8637
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3327 - accuracy: 0.8634
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3332 - accuracy: 0.8641
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3334 - accuracy: 0.8618
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3333 - accuracy: 0.8633
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 

### undersampling
### ================================================

In [21]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [22]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [24]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [25]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 1ms/step - loss: 0.6580 - accuracy: 0.6204
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6426 - accuracy: 0.6358
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6317 - accuracy: 0.6502
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.6594
Epoch 5/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6133 - accuracy: 0.6705
Epoch 6/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6058 - accuracy: 0.6744
Epoch 7/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5992 - accuracy: 0.6824
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5920 - accuracy: 0.6879
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5869 - accuracy: 0.6895
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5809 - accura

102/102 [==============================] - 0s 2ms/step - loss: 0.4643 - accuracy: 0.7742
Epoch 82/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4643 - accuracy: 0.7736
Epoch 83/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4657 - accuracy: 0.7699
Epoch 84/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4630 - accuracy: 0.7785
Epoch 85/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4660 - accuracy: 0.7720
Epoch 86/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4640 - accuracy: 0.7754
Epoch 87/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4628 - accuracy: 0.7723
Epoch 88/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4635 - accuracy: 0.7736
Epoch 89/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4630 - accuracy: 0.7729
Epoch 90/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4628 - accuracy: 

### Oversampling
### ==============================================


In [26]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [27]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [28]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6425 - accuracy: 0.6382
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6022 - accuracy: 0.6821
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5752 - accuracy: 0.7053
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5591 - accuracy: 0.7198
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7278
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 0.7286
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5273 - accuracy: 0.7326
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5161 - accuracy: 0.7431
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5078 - accuracy: 0.7491
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4996 - accura

399/399 [==============================] - 1s 1ms/step - loss: 0.4449 - accuracy: 0.7838
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4435 - accuracy: 0.7857
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4440 - accuracy: 0.7862
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4434 - accuracy: 0.7878
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4434 - accuracy: 0.7871
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4438 - accuracy: 0.7860
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4432 - accuracy: 0.7878
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4434 - accuracy: 0.7878
Epoch 89/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4428 - accuracy: 0.7863
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4428 - accuracy: 

In [29]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [30]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [32]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6479 - accuracy: 0.6244
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5998 - accuracy: 0.6844
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7191
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5279 - accuracy: 0.7456
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5058 - accuracy: 0.7578
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4947 - accuracy: 0.7629
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4894 - accuracy: 0.7644
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4858 - accuracy: 0.7652
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4834 - accuracy: 0.7649
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4823 - accura

399/399 [==============================] - 1s 2ms/step - loss: 0.4172 - accuracy: 0.8031
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4169 - accuracy: 0.8037
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4165 - accuracy: 0.8046
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4160 - accuracy: 0.8042
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.8023
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.8047
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4146 - accuracy: 0.8049
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4148 - accuracy: 0.8045
Epoch 89/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4131 - accuracy: 0.8054
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4128 - accuracy: 

### Ensemble 
### ====================================================

In [33]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [34]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [36]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [37]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [38]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [39]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [40]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6965 - accuracy: 0.5366
Epoch 2/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6679 - accuracy: 0.6026
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6538 - accuracy: 0.6262
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 0.6445
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6344 - accuracy: 0.6470
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6502
Epoch 7/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6153 - accuracy: 0.6682
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6049 - accuracy: 0.6726
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5942 - accuracy: 0.6934
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5834 - accuracy: 0.7024
Epoch 11/

98/98 [==============================] - 0s 1ms/step - loss: 0.4621 - accuracy: 0.7718
Epoch 84/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4615 - accuracy: 0.7731
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4609 - accuracy: 0.7760
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4608 - accuracy: 0.7754
Epoch 87/100
98/98 [==============================] - 0s 2ms/step - loss: 0.4614 - accuracy: 0.7757
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4584 - accuracy: 0.7709
Epoch 89/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4600 - accuracy: 0.7715
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.7750
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4605 - accuracy: 0.7741
Epoch 92/100
98/98 [==============================] - 0s 2ms/step - loss: 0.4583 - accuracy: 0.7754
Epoch 93/100


In [41]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6954 - accuracy: 0.5290
Epoch 2/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6614 - accuracy: 0.6438
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 0.6566
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6224 - accuracy: 0.6643
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6147 - accuracy: 0.6634
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6100 - accuracy: 0.6624
Epoch 7/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6038 - accuracy: 0.6707
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5980 - accuracy: 0.6771
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5943 - accuracy: 0.6790
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5885 - accuracy: 0.6874
Epoch 11/

98/98 [==============================] - 0s 1ms/step - loss: 0.4803 - accuracy: 0.7645
Epoch 84/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7635
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4833 - accuracy: 0.7629
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4794 - accuracy: 0.7632
Epoch 87/100
98/98 [==============================] - 0s 2ms/step - loss: 0.4790 - accuracy: 0.7699
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4785 - accuracy: 0.7686
Epoch 89/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4786 - accuracy: 0.7658
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7664
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7696
Epoch 92/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4780 - accuracy: 0.7718
Epoch 93/100


In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 1ms/step - loss: 0.6629 - accuracy: 0.6177
Epoch 2/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6252 - accuracy: 0.6545
Epoch 3/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6017 - accuracy: 0.6816
Epoch 4/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5904 - accuracy: 0.6888
Epoch 5/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5833 - accuracy: 0.6964
Epoch 6/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.6989
Epoch 7/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5712 - accuracy: 0.7040
Epoch 8/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5654 - accuracy: 0.7076
Epoch 9/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5592 - accuracy: 0.7126
Epoch 10/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5541 - accuracy: 0.7159
Epoch 11/

In [ ]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [ ]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)